In [150]:
import pandas as pd
import json
from tqdm import tqdm
tqdm.pandas()  # adds a progress bar to pandas

/Users/maryyoussef/opt/anaconda3/lib/python3.8/site-packages/tqdm/std.py:702: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [151]:
df = pd.read_csv('/Users/maryyoussef/Documents/Synaxarium Project/Synaxarium Project Clean Sep 22 2025.csv')

In [245]:
with open('/Users/maryyoussef/Documents/Synaxarium Project/secrets.json', 'r') as file:
        # Use json.load() to parse the JSON data directly from the file object
        keys = json.load(file)


In [152]:
len(df)

697

In [153]:
df.head()

,Unnamed: 0,Month,Link,Event,Story
0,0,Tout 1,https://www.copticchurch.net/synaxarium/1_1.ht...,Feast of El-Nayrouz(Beginning of the Blessed C...,Today is the beginning of the blessed Coptic y...
1,1,Tout 1,https://www.copticchurch.net/synaxarium/1_1.ht...,"The Martyrdom of St. Bartholomew, the Apostle.","On this day also, the holy Apostle Bartholomew..."
2,2,Tout 1,https://www.copticchurch.net/synaxarium/1_1.ht...,"The Departure of St. Melyos (Milius), the Thir...","Also on this day in the year 98 A.D., St. Mely..."
3,9,Tout 2,https://www.copticchurch.net/synaxarium/1_2.ht...,The Martyrdom of St. John the Baptist.,"On this day, the forerunner and great prophet,..."
4,10,Tout 2,https://www.copticchurch.net/synaxarium/1_2.ht...,The Martyrdom of St. Dasya the Soldier.,"On this day also, St. Dasya the soldier, who w..."


In [154]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [155]:
df.rename(columns={'Month': 'Date'}, inplace=True)

In [156]:
# Create a UID with the date and a letter index (a, b, c, ...) for each row per date
import string

def letter_indexer(df, date_col='Date'):
    # Create a letter index (a, b, c, ...) for each group of the same date
    df = df.copy()
    df['letter_idx'] = (
        df.groupby(date_col).cumcount()
        .apply(lambda x: string.ascii_lowercase[x] if x < 26 else f"z{x-25}")
    )
    df['UID'] = df[date_col].str.strip().str.replace(' ', '_').str.lower() + '_' + df['letter_idx']

    df.drop(columns=['letter_idx'], inplace=True)
    return df

df = letter_indexer(df)

In [157]:
df = df[['UID', 'Date', 'Link', 'Event', 'Story']]

In [158]:
df.sample(5)

,UID,Date,Link,Event,Story
542,paona_16_a,Paona 16,https://www.copticchurch.net/synaxarium/10_16....,The Departure of St. Abba Nofer the Anchorite.,"On this day, the ascetic father, Abba Nofer th..."
8,tout_5_a,Tout 5,https://www.copticchurch.net/synaxarium/1_5.ht...,The Martyrdom of Saint Sophia,"On this day, St. Sophia, was martyred. She use..."
493,bashons_26_a,Bashons 26,https://www.copticchurch.net/synaxarium/9_26.h...,"The Martyrdom Of St. Thomas, the Apostle.",On this day St. Thomas the apostle was martyre...
269,amshir_2_b,Amshir 2,https://www.copticchurch.net/synaxarium/6_2.ht...,"The Departure of St. Longinus, Abbot of El-Zug...","On this day also, the pure saint Anba Longinus..."
420,baramouda_17_a,Baramouda 17,https://www.copticchurch.net/synaxarium/8_17.h...,The Martyrdom of St. James the Apostle Brother...,"On this day, St. James the Apostle, the son of..."


In [ ]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://synaxarium-project-aoai.cognitiveservices.azure.com/",
    api_key=keys['aoai_subscription_key'],
)

In [160]:
prompt = '''
Extract the following structured details and return them as a Python-compatible JSON object. 
**Do not include any explanatory text or the word 'json'**—only return the JSON.

Every field listed below **must be present** in the output JSON.  
For name, if the story is about a group of people, use the group name.
If a detail is not explicitly mentioned, infer it from context where possible.  
If inference is not possible, use `"n/a"` as the value.  
For dates and ages, make the best assumption based on context.  
For lists, use an empty list `[]` if no items are present.
For places, put the city and country.  If no city is mentioned, put the country only.  Make the best assumption based on context.


{
  "name": "",   # Full name of the saint or group
  "nickname": "",   # Any commonly used nickname or short name
  "gender": "",   # Should be Male or Female 
  "ethnicity": "",  
  "titleOrJob": "",   # General title or occupation
  "rankInChurch": "",   # Pope, Patriarch, Bishop, Monk, Martyr, Deacon, etc.
  "seeOrDiocese": "",   # Alexandria, Antioch, Constantinople, etc.
  "age": "",   #Round to nearest year, should be a number
  "bcOrAd": "",   # Should be either "BC" or "AD"
  "isInBibleStory": "",   # Should be boolean True/False
  "copticDate": "",   # Coptic calendar date of commemoration
  "gregorianDate": "",   # Gregorian calendar equivalent
  "feastOrFastAssociation": "",   # e.g., Lent, Kiahk, Feast of Nativity
  "liturgicalSeason": "",   # e.g., Pascha, Pentecost
  "birthYear": "",   
  "deathYear": "",
  "placeOfBirth": "",   # City, Country or Country only
  "placeOfDeath": "",   # City, Country or Country only
  "causeOfDeath": "",   # e.g. martyrdom, natural causes, etc.
  "rulingEmperorOrKing": "",   # Historical ruler at the time
  "empireOrRegion": "",   # Roman, Byzantine, Islamic, etc.
  "warsOrPersecutions": [],   # Historical conflicts or persecutions in context
  "churchesInStory": [],   # List of churches mentioned in the story
  "commemorationInLiturgy": "",   # Whether the saint is commemorated in the liturgy
  "motherName": "",
  "fatherName": "",
  "siblingNames": [],
  "otherFamilyMembers": [],  #return a list
  "companionsOrFriends": [],  #return a list
  "connectedPeople": [],   # return list
  "intercessor": "",
  "adversary": "",
  "ally": "", 
  "familyLegacy": "",   # Influence of family line or descendants
  "locationsInStory": [],   # List of locations mentioned in the story
  "descriptiveAdjectives": [], # List of adjectives that describe the person or group
  "struggles": [],  # List of struggles or challenges faced
  "triumphOrClimax": "", 
  "conversionStory": "",   # Details if the saint converted to Christianity
  "exileOrImprisonment": "",   # Details if exiled or imprisoned
  "councilParticipation": [],   # List of councils attended (e.g., Nicaea)
  "heresiesOpposed": [],   # List of heresies opposed
  "miraclesPerformed": [],
  "persecutionsSuffered": [],
  "helpfulAnimals": [],   # List of animals that helped the person or group
  "importantVerseReference": "",   # e.g. John 3:16
  "relicsLocation": "",   # Current or historical location of relics
  "iconographySymbols": [],   # Symbols in icons (cross, palm, animals, etc.)
  "feastDayHymns": [],   # Hymns or doxologies related to the feast day
  "pilgrimageSites": [],   # Important pilgrimage sites
  "spiritualThemes": [],   # Themes like humility, courage, etc.
  "shortSummary": ""   # A brief summary of the saint's life and significance
}



'''

In [ ]:
import os
from openai import AzureOpenAI

endpoint = "https://synaxarium-project-aoai.cognitiveservices.azure.com/"
model_name = "gpt-4o-mini"
deployment = "gpt-4o-mini"

subscription_key = keys['aoai_subscription_key']
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

def extract_details(story, name, date):

    try:

        response = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant.",
                },
                {
                    "role": "user",
                    "content": f"Using the following story {story} about {name} on {date} from the Coptic Synaxarium, {prompt}",
                }
            ],
            max_tokens=4096,
            temperature=1.0,
            top_p=1.0,
            model=deployment
        )

        return response.choices[0].message.content
    except Exception as e:
        return("Error.")

In [162]:
# Get the top n longest stories by character count

top_longest = df.loc[df['Story'].str.len().sort_values(ascending=False).index]

top_longest.head()

,UID,Date,Link,Event,Story
626,abib_25_i,Abib 25,https://www.copticchurch.net/synaxarium/11_25....,"The Departure of St. Palamon, the Father of th...","On this day also, of the year 316 A.M., the sa..."
606,abib_19_a,Abib 19,https://www.copticchurch.net/synaxarium/11_19....,"The Martyrdom of Sts. Bidaba, Bishop of Qift, ...","On this day, of the year 284 A.M., which is th..."
461,bashons_7_a,Bashons 7,https://www.copticchurch.net/synaxarium/9_7.ht...,The Departure of St. Athanasius the Apostolic ...,On this day of the year 89 A.M. (373 A.D.) the...
568,paona_30_a,Paona 30,https://www.copticchurch.net/synaxarium/10_30....,"The Nativity of St. John, the Baptist.","On this day, the church celebrates the nativit..."
492,bashons_25_b,Bashons 25,https://www.copticchurch.net/synaxarium/9_25.h...,The Departure of the most honored Layman Ibrah...,On this day also of the year 1511 A.M. (1795 A...


In [163]:
top_longest['extracted_details'] = top_longest.progress_apply(
    lambda row: extract_details(row['Story'], row['Event'], row['Date']),   axis=1
)

100%|██████████| 697/697 [4:14:27<00:00, 21.90s/it]     


In [165]:
len(top_longest)

697

In [167]:
# top_longest.to_csv('all_synaxariums.csv', index=False)

In [168]:
# def json_validator(json):
#     response = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "system",
#                 "content": "You are a helpful assistant. Validate and correct a JSON object against the required schema.",
#             },
#             {
#                 "role": "user",
#                 "content": f'''
#           You are a JSON validation assistant.

#           Given the following JSON object, check if it matches the required schema below.  
#           If the structure is correct, return the JSON object as-is.  
#           If the structure is incorrect (missing fields, wrong types, extra fields, etc.), return a corrected JSON object that matches the schema, 
#           filling missing or invalid fields with "n/a" or an empty list as appropriate.  
#           **Do not include any explanation or extra text—return only the corrected JSON.**

#                         Required schema:
#                         {{
#   "name": "",   # Full name of the saint or group
#   "nickname": "",   # Any commonly used nickname or short name
#   "gender": "",   # Should be Male or Female 
#   "ethnicity": "",  
#   "titleOrJob": "",   # General title or occupation
#   "rankInChurch": "",   # Pope, Patriarch, Bishop, Monk, Martyr, Deacon, etc.
#   "seeOrDiocese": "",   # Alexandria, Antioch, Constantinople, etc.
#   "age": "",
#   "bcOrAd": "",   # Should be either "BC" or "AD"
#   "isInBibleStory": "",   # Should be boolean True/False
#   "copticDate": "",   # Coptic calendar date of commemoration
#   "gregorianDate": "",   # Gregorian calendar equivalent
#   "feastOrFastAssociation": "",   # e.g., Lent, Kiahk, Feast of Nativity
#   "liturgicalSeason": "",   # e.g., Pascha, Pentecost
#   "birthYear": "",   
#   "deathYear": "",
#   "placeOfBirth": "",   # City, Country or Country only (use common spelling if spelling differs)
#   "placeOfDeath": "",   # City, Country or Country only (use common spelling if spelling differs)
#   "causeOfDeath": "",   # e.g. martyrdom, natural causes, etc.
#   "rulingEmperorOrKing": "",   # Historical ruler at the time
#   "empireOrRegion": "",   # Roman, Byzantine, Islamic, etc.
#   "warsOrPersecutions": [],   # Historical conflicts or persecutions in context
#   "churchesInStory": [],   # List of churches mentioned in the story
#   "commemorationInLiturgy": "",   # Whether the saint is commemorated in the liturgy
#   "motherName": "",
#   "fatherName": "",
#   "siblingNames": [],
#   "otherFamilyMembers": [],
#   "companionsOrFriends": [],
#   "connectedPeople": [],   # Edge list with relationship as an attribute
#   "intercessor": "",
#   "adversary": "",
#   "ally": "", 
#   "familyLegacy": "",   # Influence of family line or descendants
#   "locationsInStory": [],   # List of locations mentioned in the story
#   "descriptiveAdjectives": [], # List of adjectives that describe the person or group
#   "struggles": [],  # List of struggles or challenges faced
#   "triumphOrClimax": "", 
#   "conversionStory": "",   # Details if the saint converted to Christianity
#   "exileOrImprisonment": "",   # Details if exiled or imprisoned
#   "councilParticipation": [],   # List of councils attended (e.g., Nicaea)
#   "heresiesOpposed": [],   # List of heresies opposed
#   "miraclesPerformed": [],
#   "persecutionsSuffered": [],
#   "helpfulAnimals": [],   # List of animals that helped the person or group
#   "importantVerseReference": "",   # e.g. John 3:16
#   "relicsLocation": "",   # Current or historical location of relics
#   "iconographySymbols": [],   # Symbols in icons (cross, palm, animals, etc.)
#   "feastDayHymns": [],   # Hymns or doxologies related to the feast day
#   "pilgrimageSites": [],   # Important pilgrimage sites
#   "spiritualThemes": [],   # Themes like humility, courage, etc.
#   "shortSummary": ""   # A brief summary of the saint's life and significance
# }}

#                           JSON to check:
#                           {json} ''',
#             }
#         ],
#         max_tokens=4096,
#         temperature=1.0,
#         top_p=1.0,
#         model=deployment
#     )

#     return response.choices[0].message.content

In [169]:
def make_json(json_str):
    try:
        json_out = json.loads(json_str)
        print("Good json!")
        return json_out
        
    except Exception:
        print("Bad json!")
        return "Bad json!"
        

In [170]:
top_longest['extracted_details'] = top_longest['extracted_details'].apply(lambda x: make_json(x))

Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Bad json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Bad json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
Good json!
G

In [171]:
top_longest = top_longest[top_longest['extracted_details'] != "Bad json!"]
len(top_longest)

641

In [172]:
# Expand the extracted_details column into separate columns
expanded = pd.json_normalize(top_longest['extracted_details'])

# Concatenate with the original columns you want to keep
result = pd.concat([top_longest[['UID', 'Date', 'Link', 'Story']].reset_index(drop=True), expanded], axis=1)

# Display the expanded DataFrame
result.head()

,UID,Date,Link,Story,name,nickname,gender,ethnicity,titleOrJob,rankInChurch,...,miraclesPerformed,persecutionsSuffered,helpfulAnimals,importantVerseReference,relicsLocation,iconographySymbols,feastDayHymns,pilgrimageSites,spiritualThemes,shortSummary
0,abib_25_i,Abib 25,https://www.copticchurch.net/synaxarium/11_25....,"On this day also, of the year 316 A.M., the sa...",St. Palamon,Anba Palamon,Male,Coptic,Anchorite,Saint,...,"[Healing, Companion with beasts]",[],[Raven],Psalm 54,Church of St. Palamon in El-Kasre,"[Cross, Desert, Raven]",[],[Monastery of St. Palamon],"[asceticism, perseverance, forgiveness]",St. Palamon was an anchorite renowned for his ...
1,abib_19_a,Abib 19,https://www.copticchurch.net/synaxarium/11_19....,"On this day, of the year 284 A.M., which is th...","Sts. Bidaba, Andrew, and Christodoulas",El-Gawhary,Male,Coptic,Bishop,Bishop,...,"[Healed a handicapped man, Various signs and w...",[Tortured for Christianity],[],Matthew 10:32-33,n/a,"[Crown, Cross]",[],[],"[Courage, Faith, Martyrdom]",St. Bidaba was a devoted Christian bishop know...
2,bashons_7_a,Bashons 7,https://www.copticchurch.net/synaxarium/9_7.ht...,On this day of the year 89 A.M. (373 A.D.) the...,St. Athanasius the Apostolic,Athanasius,Male,Coptic,Pope,Pope,...,[],"[Five exiles, false accusations]",[],n/a,"Alexandria, Egypt","[cross, palm branch]",[],[n/a],"[faith, resilience, sacrifice]","St. Athanasius, the 20th Pope of Alexandria, w..."
3,paona_30_a,Paona 30,https://www.copticchurch.net/synaxarium/10_30....,"On this day, the church celebrates the nativit...",St. John the Baptist,John,Male,Jewish,Prophet,Martyr,...,[],[Beheading by Herod],[],Matthew 3:16-17,n/a,"[camel's hair, locusts, honey]",[],[],"[repentance, humility]",St. John the Baptist was a prophet who prepare...
4,bashons_25_b,Bashons 25,https://www.copticchurch.net/synaxarium/9_25.h...,On this day also of the year 1511 A.M. (1795 A...,Ibrahim El-Gohari,The Sultan of the Copts,Male,Coptic,"Layman, Chief Scribe",n/a,...,[],[Persecution of Copts under Hasan Qubtan],[],n/a,Tomb beside the church of St. George in Old Cairo,[],[],[],"[charity, endurance, faith]",Ibrahim El-Gohari was a prominent Coptic layma...


In [173]:
# Replace 'n/a' and blank strings with None (null)
result = result.replace({'n/a': None, '': None})

# Display the expanded DataFrame
result.head()

,UID,Date,Link,Story,name,nickname,gender,ethnicity,titleOrJob,rankInChurch,...,miraclesPerformed,persecutionsSuffered,helpfulAnimals,importantVerseReference,relicsLocation,iconographySymbols,feastDayHymns,pilgrimageSites,spiritualThemes,shortSummary
0,abib_25_i,Abib 25,https://www.copticchurch.net/synaxarium/11_25....,"On this day also, of the year 316 A.M., the sa...",St. Palamon,Anba Palamon,Male,Coptic,Anchorite,Saint,...,"[Healing, Companion with beasts]",[],[Raven],Psalm 54,Church of St. Palamon in El-Kasre,"[Cross, Desert, Raven]",[],[Monastery of St. Palamon],"[asceticism, perseverance, forgiveness]",St. Palamon was an anchorite renowned for his ...
1,abib_19_a,Abib 19,https://www.copticchurch.net/synaxarium/11_19....,"On this day, of the year 284 A.M., which is th...","Sts. Bidaba, Andrew, and Christodoulas",El-Gawhary,Male,Coptic,Bishop,Bishop,...,"[Healed a handicapped man, Various signs and w...",[Tortured for Christianity],[],Matthew 10:32-33,None,"[Crown, Cross]",[],[],"[Courage, Faith, Martyrdom]",St. Bidaba was a devoted Christian bishop know...
2,bashons_7_a,Bashons 7,https://www.copticchurch.net/synaxarium/9_7.ht...,On this day of the year 89 A.M. (373 A.D.) the...,St. Athanasius the Apostolic,Athanasius,Male,Coptic,Pope,Pope,...,[],"[Five exiles, false accusations]",[],None,"Alexandria, Egypt","[cross, palm branch]",[],[n/a],"[faith, resilience, sacrifice]","St. Athanasius, the 20th Pope of Alexandria, w..."
3,paona_30_a,Paona 30,https://www.copticchurch.net/synaxarium/10_30....,"On this day, the church celebrates the nativit...",St. John the Baptist,John,Male,Jewish,Prophet,Martyr,...,[],[Beheading by Herod],[],Matthew 3:16-17,None,"[camel's hair, locusts, honey]",[],[],"[repentance, humility]",St. John the Baptist was a prophet who prepare...
4,bashons_25_b,Bashons 25,https://www.copticchurch.net/synaxarium/9_25.h...,On this day also of the year 1511 A.M. (1795 A...,Ibrahim El-Gohari,The Sultan of the Copts,Male,Coptic,"Layman, Chief Scribe",None,...,[],[Persecution of Copts under Hasan Qubtan],[],None,Tomb beside the church of St. George in Old Cairo,[],[],[],"[charity, endurance, faith]",Ibrahim El-Gohari was a prominent Coptic layma...


In [174]:
locsInStory = result[['UID', 'locationsInStory']].explode('locationsInStory')
locsInStory = locsInStory[~locsInStory['locationsInStory'].apply(lambda x: isinstance(x, dict))]
locsInStory.drop_duplicates(inplace=True)
locsInStory.dropna(inplace=True)
locsInStory.reset_index(drop=True, inplace=True)
locsInStory['Type'] = 'locationInStory'
locsInStory.rename(columns={'locationsInStory': 'Place'}, inplace=True)
result.drop(columns=['locationsInStory'], inplace=True)
locsInStory



,UID,Place,Type
0,abib_25_i,"El-Kasre, Egypt",locationInStory
1,abib_25_i,"Nagaa-Hammady, Egypt",locationInStory
2,abib_25_i,Mountains of Egypt,locationInStory
3,abib_19_a,Qift,locationInStory
4,abib_19_a,Armant,locationInStory
...,...,...,...
1182,bashons_6_c,"El-Bandarah, Egypt",locationInStory
1183,abib_15_c,"Soul, Egypt",locationInStory
1184,hator_16_b,"El-Zaher, Cairo, Egypt",locationInStory
1185,paona_4_d,Mount El-Amoud,locationInStory


In [175]:
pilgrimageSites = result[['UID', 'pilgrimageSites']].explode('pilgrimageSites')
pilgrimageSites.drop_duplicates(inplace=True)
pilgrimageSites.dropna(inplace=True)
pilgrimageSites.reset_index(drop=True, inplace=True)
pilgrimageSites['Type'] = 'pilgrimageSites'
pilgrimageSites.rename(columns={'pilgrimageSites': 'Place'}, inplace=True)
result.drop(columns=['pilgrimageSites'], inplace=True)
pilgrimageSites

,UID,Place,Type
0,abib_25_i,Monastery of St. Palamon,pilgrimageSites
1,bashons_7_a,n/a,pilgrimageSites
2,bashons_12_a,Church of St. Demiana,pilgrimageSites
3,hator_15_a,Monastery of Saint Mina,pilgrimageSites
4,toba_22_a,monastery of St. Anthony the Great,pilgrimageSites
5,baramouda_30_a,Alexandria,pilgrimageSites
6,baramouda_27_a,"Bokalia, Alexandria",pilgrimageSites
7,hator_29_a,Tomb of St. Mark,pilgrimageSites
8,paona_3_e,His tomb,pilgrimageSites
9,baramouda_23_a,Church of St. George in Lydda,pilgrimageSites


In [176]:
churchesInStory = result[['UID', 'churchesInStory']].explode('churchesInStory')
churchesInStory.drop_duplicates(inplace=True)
churchesInStory.dropna(inplace=True)
churchesInStory.reset_index(drop=True, inplace=True)
churchesInStory['Type'] = 'churchInStory'
churchesInStory.rename(columns={'churchesInStory': 'Place'}, inplace=True)
result.drop(columns=['churchesInStory'], inplace=True)
churchesInStory

,UID,Place,Type
0,abib_25_i,Church of St. Palamon,churchInStory
1,bashons_25_b,St. Mark Cathedral,churchInStory
2,bashons_25_b,Church of the Virgin in Haret-Zoweila,churchInStory
3,bashons_25_b,Church of Abu Saifain in the Monastery of St. ...,churchInStory
4,bashons_25_b,Church of the Virgin in Haret El-Roum,churchInStory
...,...,...,...
262,paona_28_b,Church of Anba Sarabamoun,churchInStory
263,kiahk_16_c,Church of St. James the Persian,churchInStory
264,mesra_4_b,Church of the great St. Anthony,churchInStory
265,hator_16_b,Church of St. Abu-Nofer,churchInStory


In [177]:
placeOfBirth = result[['UID', 'placeOfBirth']]
placeOfBirth.dropna(inplace=True)
placeOfBirth.drop_duplicates(inplace=True)
placeOfBirth.reset_index(drop=True, inplace=True)
placeOfBirth['Type'] = 'placeOfBirth'
placeOfBirth.rename(columns={'placeOfBirth': 'Place'}, inplace=True)
result.drop(columns=['placeOfBirth'], inplace=True)
placeOfBirth

<ipython-input-177-5892fb9366f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeOfBirth.dropna(inplace=True)
<ipython-input-177-5892fb9366f7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeOfBirth.drop_duplicates(inplace=True)
<ipython-input-177-5892fb9366f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeOfBirth['Type'] = 'placeOfBirth'
/Users/maryyoussef/op

,UID,Place,Type
0,abib_19_a,"Armant, Egypt",placeOfBirth
1,bashons_25_b,"Kalube, Egypt",placeOfBirth
2,baramhat_28_b,"Gawli, Egypt",placeOfBirth
3,bashons_12_a,"El-Zaafran, Egypt",placeOfBirth
4,mesra_25_b,"El-Mahalla El-Koubra, Egypt",placeOfBirth
...,...,...,...
250,kiahk_24_c,Ethiopia,placeOfBirth
251,kiahk_16_b,"El-Fayyoum, Egypt",placeOfBirth
252,baramhat_20_b,"Bethany, Israel",placeOfBirth
253,bashons_6_c,"El-Bandarah, Egypt",placeOfBirth


In [178]:
placeOfDeath = result[['UID', 'placeOfDeath']]
placeOfDeath.dropna(inplace=True)
placeOfDeath.drop_duplicates(inplace=True)
placeOfDeath.reset_index(drop=True, inplace=True)
placeOfDeath['Type'] = 'placeOfDeath'
placeOfDeath.rename(columns={'placeOfDeath': 'Place'}, inplace=True)
result.drop(columns=['placeOfDeath'], inplace=True)
placeOfDeath

<ipython-input-178-0297acb0ecd7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeOfDeath.dropna(inplace=True)
<ipython-input-178-0297acb0ecd7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeOfDeath.drop_duplicates(inplace=True)
<ipython-input-178-0297acb0ecd7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  placeOfDeath['Type'] = 'placeOfDeath'
/Users/maryyoussef/op

,UID,Place,Type
0,abib_25_i,"El-Kasre, Egypt",placeOfDeath
1,abib_19_a,"Esna, Egypt",placeOfDeath
2,bashons_7_a,"Alexandria, Egypt",placeOfDeath
3,paona_30_a,"Machaerus, Jordan",placeOfDeath
4,bashons_25_b,"Cairo, Egypt",placeOfDeath
...,...,...,...
285,baramouda_11_b,"Gaza, n/a",placeOfDeath
286,bashons_13_b,"Alexandria, Egypt",placeOfDeath
287,bashons_6_c,"El-Bandarah, Egypt",placeOfDeath
288,abib_15_c,"Soul, Egypt",placeOfDeath


In [179]:
relicsLocation = result[['UID', 'relicsLocation']]
relicsLocation.dropna(inplace=True)
relicsLocation.drop_duplicates(inplace=True)
relicsLocation.reset_index(drop=True, inplace=True)
relicsLocation['Type'] = 'relicsLocation'
relicsLocation.rename(columns={'relicsLocation': 'Place'}, inplace=True)
result.drop(columns=['relicsLocation'], inplace=True)
relicsLocation

<ipython-input-179-3ab4d366a0db>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relicsLocation.dropna(inplace=True)
<ipython-input-179-3ab4d366a0db>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relicsLocation.drop_duplicates(inplace=True)
<ipython-input-179-3ab4d366a0db>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relicsLocation['Type'] = 'relicsLocation'
/Users/maryyo

,UID,Place,Type
0,abib_25_i,Church of St. Palamon in El-Kasre,relicsLocation
1,bashons_7_a,"Alexandria, Egypt",relicsLocation
2,bashons_25_b,Tomb beside the church of St. George in Old Cairo,relicsLocation
3,bashons_23_b,Tomb beside the church of St. George in Old Cairo,relicsLocation
4,baramhat_28_b,"El-Azbakiah, Cairo, Egypt",relicsLocation
...,...,...,...
135,toba_20_b,"St. John's Church, Rome",relicsLocation
136,hator_5_b,"Shotb, Egypt",relicsLocation
137,abib_29_a,Constantinople,relicsLocation
138,baba_30_a,Alexandria,relicsLocation


In [180]:
allLocations = pd.concat([locsInStory, pilgrimageSites, churchesInStory, placeOfBirth, placeOfDeath, relicsLocation], ignore_index=True    )
allLocations.drop_duplicates(subset=['UID', 'Place'], inplace=True)
allLocations['Place'] = allLocations['Place'].str.replace('n/a', '')

In [186]:
allLocations = allLocations[allLocations['Place'] != '']

In [189]:
allLocations.sample(10)

,UID,Place,Type
1562,toba_30_a,"Antioch, Syria",placeOfBirth
1480,amshir_15_b,First Church Dedicated to the Forty Martyrs of...,churchInStory
1490,paona_5_c,Church of St. Boctor,churchInStory
254,baramouda_24_a,Desert of Scetis,locationInStory
171,bashons_21_a,Athens,locationInStory
1932,baramouda_4_a,"Scetis, Egypt",placeOfDeath
835,bashons_4_c,Old Cairo,locationInStory
1303,hator_18_a,great church built for Atrasis and Yoana,churchInStory
774,baramouda_26_b,El-Nastour monastery,locationInStory
2064,hator_17_a,"Church of the Apostles, Constantinople",relicsLocation


In [ ]:
from googlemaps import Client as GoogleMaps

key = keys['google_maps_key']

gmaps = GoogleMaps(key)
    

In [191]:
def get_coords(place):
    if isinstance(place, list):
        try:
            geocode_result_place = [gmaps.geocode(i) for i in place]
            print(gmaps.geocode(i[0]['formatted_address']))
            if 'USA' in gmaps.geocode(i[0]['formatted_address']):
                print('In the USA')
                return {}
            else:
                
                geocode_result_place = [i[0]['address_components'][0]['long_name'] for i in geocode_result_place]

                geocode_result_coord = [gmaps.geocode(i)for i in place]
                geocode_result_coord = [i[0]['geometry']['location'] for i in geocode_result_coord]
            
                print(f"{place} | {geocode_result_place} | {geocode_result_coord}\n")
            
                return geocode_result_coord
        except IndexError:
            return None
    else:
        try:

            if 'USA' in gmaps.geocode(place)[0]['formatted_address']:
                return {}
            else:   
                geocode_result_place = gmaps.geocode(place)[0]['address_components'][0]['long_name']
                geocode_result_coord = gmaps.geocode(place)[0]['geometry']['location']
                print(f"{place} | {geocode_result_place} | {geocode_result_coord}\n")

                return geocode_result_coord    
        except IndexError:
             return None


In [192]:
allLocations['Coords'] = allLocations['Place'].apply(lambda x: get_coords(x) if x else None)

El-Kasre, Egypt | 20 | {'lat': 30.105948, 'lng': 31.253717}

Nagaa-Hammady, Egypt | Nagaa Hammadi | {'lat': 26.0451551, 'lng': 32.2427609}

Mountains of Egypt | Egypt | {'lat': 26.820553, 'lng': 30.802498}

Qift | Qift | {'lat': 26.0002314, 'lng': 32.8159277}

Armant | Armant | {'lat': 25.620661, 'lng': 32.54526269999999}

Esna | Esna | {'lat': 25.2979411, 'lng': 32.554263}

Bahgourah, Egypt | Bahgourah | {'lat': 26.0474765, 'lng': 32.2143323}

Alexandria | Alexandria | {'lat': 31.2000924, 'lng': 29.9187387}

Treves, France | Trèves | {'lat': 44.078389, 'lng': 3.388931}

Rome | Rome | {'lat': 41.8967068, 'lng': 12.4822025}

Arles | Arles | {'lat': 43.676647, 'lng': 4.6277769}

Milan | Milan | {'lat': 45.468503, 'lng': 9.182402699999999}

Desert of Ziphana | 9VPJ+V67 | {'lat': 15.3871647, 'lng': 78.8805326}

Jordan River | Jordan River | {'lat': 32.46708430667223, 'lng': 35.56573872483355}

Machaerus | Machaerus | {'lat': 31.5646344, 'lng': 35.6374837}

Cairo | Cairo | {'lat': 30.044419

<ipython-input-192-63a35c8fcc23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  allLocations['Coords'] = allLocations['Place'].apply(lambda x: get_coords(x) if x else None)


In [193]:
allLocations = allLocations[allLocations['Coords'] != {}]

In [194]:
allLocations.dropna(subset=['Coords'], inplace=True)
allLocations.reset_index(drop=True, inplace=True)   

# Pull out lat and lng
allLocations["lat"] = allLocations["Coords"].apply(lambda x: x["lat"])
allLocations["lng"] = allLocations["Coords"].apply(lambda x: x["lng"])

allLocations.drop(columns=['Coords'], inplace=True)

allLocations.head(10)

,UID,Place,Type,lat,lng
0,abib_25_i,"El-Kasre, Egypt",locationInStory,30.105948,31.253717
1,abib_25_i,"Nagaa-Hammady, Egypt",locationInStory,26.045155,32.242761
2,abib_25_i,Mountains of Egypt,locationInStory,26.820553,30.802498
3,abib_19_a,Qift,locationInStory,26.000231,32.815928
4,abib_19_a,Armant,locationInStory,25.620661,32.545263
5,abib_19_a,Esna,locationInStory,25.297941,32.554263
6,abib_19_a,"Bahgourah, Egypt",locationInStory,26.047476,32.214332
7,bashons_7_a,Alexandria,locationInStory,31.200092,29.918739
8,bashons_7_a,"Treves, France",locationInStory,44.078389,3.388931
9,bashons_7_a,Rome,locationInStory,41.896707,12.482202


In [195]:
result['birthYear'] = result['birthYear'].fillna('').str.replace(r"\D", "", regex=True)
result['deathYear'] = result['deathYear'].fillna('').str.replace(r"\D", "", regex=True)

In [196]:
result['gender'] = result['gender'].str.title()
result['ethnicity'] = result['ethnicity'].str.title()
result['titleOrJob'] = result['titleOrJob'].str.title()

# Standardize 'bcOrAd' column to uppercase and remove periods
result['bcOrAd'] = result['bcOrAd'].str.replace('.', '', regex=False).str.upper()

In [197]:
importantVerseReference = result[['UID', 'importantVerseReference']]
importantVerseReference.dropna(inplace=True)
importantVerseReference.drop_duplicates(inplace=True)
importantVerseReference.reset_index(drop=True, inplace=True)
importantVerseReference['importantVerseReference'] = importantVerseReference['importantVerseReference'].apply(lambda x : x.split(','))
importantVerseReference = importantVerseReference.explode('importantVerseReference')
importantVerseReference.dropna(inplace=True)
importantVerseReference.reset_index(drop=True, inplace=True)
importantVerseReference

<ipython-input-197-c3e7f22eb1c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  importantVerseReference.dropna(inplace=True)
<ipython-input-197-c3e7f22eb1c9>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  importantVerseReference.drop_duplicates(inplace=True)
<ipython-input-197-c3e7f22eb1c9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  importantVerseReference['importantVer

,UID,importantVerseReference
0,abib_25_i,Psalm 54
1,abib_19_a,Matthew 10:32-33
2,paona_30_a,Matthew 3:16-17
3,kiahk_6_a,Matthew 17:20
4,toba_22_a,Matthew 19:21
...,...,...
119,bashons_13_a,Romans 16
120,baramhat_21_b,John 12:8
121,abib_12_a,Revelation 12:7-9
122,baramouda_15_b,Acts 21:10-11


In [204]:
import requests

def get_bible_verse(verse):
    try:
        verse = verse.strip()
        response = requests.get(f"https://bible-api.com/{verse.replace(' ', '%20')}")
        data = response.json()
        return data['text'] if 'text' in data else f"Verse not found: {verse}"
    except Exception as e:
        return f"Verse not found: {verse}"

In [205]:
importantVerseReference

,UID,importantVerseReference
0,abib_25_i,Psalm 54
1,abib_19_a,Matthew 10:32-33
2,paona_30_a,Matthew 3:16-17
3,kiahk_6_a,Matthew 17:20
4,toba_22_a,Matthew 19:21
...,...,...
119,bashons_13_a,Romans 16
120,baramhat_21_b,John 12:8
121,abib_12_a,Revelation 12:7-9
122,baramouda_15_b,Acts 21:10-11


In [206]:
# Apply get_bible_verse to each item in the list, join results with space, or None if empty
importantVerseReference['importantVerseReference_full'] = importantVerseReference['importantVerseReference'].apply(
    lambda verses: get_bible_verse(verses))

importantVerseReference['importantVerseReference_full'] = importantVerseReference['importantVerseReference_full'].apply(lambda x: x.replace('\n', ' ') if x else None)

importantVerseReference['importantVerseReference_full'] = importantVerseReference['importantVerseReference_full'].apply(lambda x: x.strip() if x else None)


In [207]:
companionsOrFriends = result[['UID', 'companionsOrFriends']].explode('companionsOrFriends')
companionsOrFriends.dropna(inplace=True)
companionsOrFriends.drop_duplicates(inplace=True)
companionsOrFriends.reset_index(drop=True, inplace=True)
companionsOrFriends['Type'] = 'companionsOrFriends'
companionsOrFriends.rename(columns={'companionsOrFriends': 'Relationship'}, inplace=True)
result.drop(columns=['companionsOrFriends'], inplace=True)

companionsOrFriends

,UID,Relationship,Type
0,abib_25_i,St. Talasoun,companionsOrFriends
1,abib_25_i,St. Pachomius,companionsOrFriends
2,abib_19_a,Andrew,companionsOrFriends
3,abib_19_a,Christodoulas,companionsOrFriends
4,bashons_7_a,St. Antonios,companionsOrFriends
...,...,...,...
272,mesra_3_b,St. Mark the Apostle,companionsOrFriends
273,mesra_3_b,Bishop Anianus,companionsOrFriends
274,bashons_30_b,St. Paul,companionsOrFriends
275,baramouda_15_b,St. Paul,companionsOrFriends


In [208]:
motherName = result[['UID', 'motherName']].explode('motherName')
motherName.dropna(inplace=True)
motherName.drop_duplicates(inplace=True)
motherName.reset_index(drop=True, inplace=True)
motherName['Type'] = 'motherName'
motherName.rename(columns={'motherName': 'Relationship'}, inplace=True)
result.drop(columns=['motherName'], inplace=True)

motherName

,UID,Relationship,Type
0,paona_30_a,Elizabeth,motherName
1,hator_15_a,Euphemia,motherName
2,baramhat_27_b,Sarah,motherName
3,baramouda_30_a,Mary,motherName
4,baramouda_27_a,Mary,motherName
5,hator_29_a,Sophia,motherName
6,kiahk_29_a,St. Mary,motherName
7,baramouda_23_a,Theobaste,motherName
8,hator_17_a,Anthosa,motherName
9,baramhat_28_a,Helena,motherName


In [209]:
fatherName = result[['UID', 'fatherName']].explode('fatherName')
fatherName.dropna(inplace=True)
fatherName.drop_duplicates(inplace=True)
fatherName.reset_index(drop=True, inplace=True)
fatherName['Type'] = 'fatherName'
fatherName.rename(columns={'fatherName': 'Relationship'}, inplace=True)
result.drop(columns=['fatherName'], inplace=True)

fatherName

,UID,Relationship,Type
0,paona_30_a,Zacharias,fatherName
1,bashons_25_b,Yousef El-Gohari,fatherName
2,bashons_23_b,Yousef El-Gohari,fatherName
3,bashons_12_a,Mark,fatherName
4,hator_15_a,Eudoxius,fatherName
...,...,...,...
95,baramhat_21_b,Joseph,fatherName
96,hator_20_b,John,fatherName
97,paona_11_b,Numerianus,fatherName
98,paona_9_b,Numerianus,fatherName


In [210]:
siblingNames = result[['UID', 'siblingNames']].explode('siblingNames')
siblingNames.dropna(inplace=True)
siblingNames.drop_duplicates(inplace=True)
siblingNames.reset_index(drop=True, inplace=True)
siblingNames['Type'] = 'siblingNames'
siblingNames.rename(columns={'siblingNames': 'Relationship'}, inplace=True)
result.drop(columns=['siblingNames'], inplace=True)

siblingNames

,UID,Relationship,Type
0,bashons_25_b,Girgis El-Gohari,siblingNames
1,bashons_23_b,Girgis El-Gohari,siblingNames
2,mesra_16_b,n/a,siblingNames
3,baba_20_a,n/a,siblingNames
4,toba_22_a,sister,siblingNames
...,...,...,...
80,bashons_27_b,Mary,siblingNames
81,bashons_27_b,Martha,siblingNames
82,hator_3_b,Irene,siblingNames
83,mesra_26_b,Thecla,siblingNames


In [211]:
otherFamilyMembers = result[['UID', 'otherFamilyMembers']].explode('otherFamilyMembers')
otherFamilyMembers.dropna(inplace=True)
otherFamilyMembers.drop_duplicates(inplace=True)
otherFamilyMembers.reset_index(drop=True, inplace=True)
otherFamilyMembers['Type'] = 'otherFamilyMembers'
otherFamilyMembers.rename(columns={'otherFamilyMembers': 'Relationship'}, inplace=True)
result.drop(columns=['otherFamilyMembers'], inplace=True)

otherFamilyMembers

,UID,Relationship,Type
0,abib_19_a,Andrew (cousin),otherFamilyMembers
1,bashons_25_b,Wife,otherFamilyMembers
2,bashons_25_b,Son - Joseph,otherFamilyMembers
3,bashons_25_b,Daughter - Demiana,otherFamilyMembers
4,bashons_23_b,Wife,otherFamilyMembers
...,...,...,...
79,baramouda_24_b,Mother,otherFamilyMembers
80,kiahk_24_b,daughter,otherFamilyMembers
81,baba_19_b,his wife,otherFamilyMembers
82,baramhat_9_b,Amrata (wife),otherFamilyMembers


In [212]:
adversary = result[['UID', 'adversary']].explode('adversary')
adversary.dropna(inplace=True)
adversary.drop_duplicates(inplace=True)
adversary.reset_index(drop=True, inplace=True)
adversary['Type'] = 'adversary'
adversary.rename(columns={'adversary': 'Relationship'}, inplace=True)
result.drop(columns=['adversary'], inplace=True)

adversary

,UID,Relationship,Type
0,abib_25_i,The Devil,adversary
1,abib_19_a,Arianus,adversary
2,bashons_7_a,Arius,adversary
3,paona_30_a,Herod Antipas,adversary
4,bashons_25_b,Hasan Qubtan,adversary
...,...,...,...
334,amshir_10_d,Persian emperor,adversary
335,hator_16_a,sin,adversary
336,baramhat_21_c,Jesus Christ,adversary
337,hator_16_c,Maximus the prince,adversary


In [213]:
ally = result[['UID', 'ally']].explode('ally')
ally.dropna(inplace=True)
ally.drop_duplicates(inplace=True)
ally.reset_index(drop=True, inplace=True)
ally['Type'] = 'ally'
ally.rename(columns={'ally': 'Relationship'}, inplace=True)
result.drop(columns=['ally'], inplace=True)

ally

,UID,Relationship,Type
0,abib_25_i,St. Talasoun,ally
1,bashons_7_a,Pope Alexandros,ally
2,paona_30_a,Jesus Christ,ally
3,bashons_25_b,Pope Youhanna (John) XVIII,ally
4,bashons_23_b,Pope John (Youhanna) the Eighteenth,ally
...,...,...,...
156,amshir_15_b,St. Basil the Great,ally
157,hator_16_a,Lord,ally
158,abib_29_a,Emperor Constantine,ally
159,toba_14_b,St. Domadius,ally


In [214]:
relationships = pd.concat([companionsOrFriends, motherName, fatherName, siblingNames, otherFamilyMembers, adversary, ally])
relationships.reset_index(drop=True, inplace=True)

In [218]:
relationships.sample(10)

,UID,Relationship,Type
583,hator_5_a,the woman's son,otherFamilyMembers
69,tout_14_a,Abba Yoanis,companionsOrFriends
444,toba_6_c,St. Kisarion,siblingNames
684,abib_3_b,Nestorius,adversary
925,baramhat_1_b,Maximianus,adversary
11,toba_22_a,Anba Serapion,companionsOrFriends
674,kiahk_4_a,idols' priests,adversary
902,mesra_29_a,Arianus,adversary
665,toba_6_d,Emperor Valens,adversary
993,paona_2_b,Anba Yusab El-Abbah,ally


In [219]:
descriptiveAdjectives = result[['UID', 'descriptiveAdjectives']].explode('descriptiveAdjectives')
descriptiveAdjectives.dropna(inplace=True)
descriptiveAdjectives.drop_duplicates(inplace=True)
descriptiveAdjectives.reset_index(drop=True, inplace=True)
descriptiveAdjectives['descriptiveAdjectives'] = descriptiveAdjectives['descriptiveAdjectives'].str.title()
result.drop(columns=['descriptiveAdjectives'], inplace=True)

descriptiveAdjectives

,UID,descriptiveAdjectives
0,abib_25_i,blessed
1,abib_25_i,righteous
2,abib_25_i,compassionate
3,abib_19_a,great
4,abib_19_a,holy
...,...,...
1523,paona_1_b,martyr
1524,tout_20_b,virgin
1525,tout_20_b,martyr
1526,paona_4_b,Righteous


In [ ]:
iconographySymbols = result[['UID', 'iconographySymbols']].explode('iconographySymbols')
iconographySymbols.dropna(inplace=True)
iconographySymbols = iconographySymbols[iconographySymbols['iconographySymbols'] != 'n/a']
iconographySymbols.drop_duplicates(inplace=True)
iconographySymbols.reset_index(drop=True, inplace=True)

result.drop(columns=['iconographySymbols'], inplace=True)

iconographySymbols


In [232]:
spiritualThemes = result[['UID', 'spiritualThemes']].explode('spiritualThemes')
spiritualThemes.dropna(inplace=True)
spiritualThemes.drop_duplicates(inplace=True)
spiritualThemes.reset_index(drop=True, inplace=True)
spiritualThemes['spiritualThemes'] = spiritualThemes['spiritualThemes'].str.lower()
result.drop(columns=['spiritualThemes'], inplace=True)

spiritualThemes

,UID,spiritualThemes
0,abib_25_i,asceticism
1,abib_25_i,perseverance
2,abib_25_i,forgiveness
3,abib_19_a,courage
4,abib_19_a,faith
...,...,...
1374,paona_1_b,faith
1375,tout_20_b,sacrifice
1376,tout_20_b,faith
1377,paona_4_b,faith


In [233]:
result.sample(5)

,UID,Date,Link,Story,name,nickname,gender,ethnicity,titleOrJob,rankInChurch,...,conversionStory,exileOrImprisonment,councilParticipation,heresiesOpposed,miraclesPerformed,persecutionsSuffered,helpfulAnimals,importantVerseReference,feastDayHymns,shortSummary
471,abib_12_b,Abib 12,https://www.copticchurch.net/synaxarium/11_12....,"On this day also, St. Abba Hour, was martyred....",St. Abba Hour El-Siriakousy,None,Male,None,Iron Worker,Martyr,...,Governor and his family converted to Christianity,None,[],[],[Healing of his wounds],"[Squeezed by the wheel, Crucified head down, B...",[],None,[],St. Abba Hour was a martyr from Siriakous who ...
591,amshir_25_b,Amshir 25,https://www.copticchurch.net/synaxarium/6_25.h...,On this day also is the commemoration of the m...,St. Quona and St. Mina,None,Male,Coptic,Deacon,Martyr,...,None,None,[],[],[],[martyrdom],[],None,[],St. Quona of Rome and St. Mina of Cyprus are c...
410,baba_28_a,Baba 28,https://www.copticchurch.net/synaxarium/2_28.h...,"On this day of the year 351 A.D., the great sa...",Saints Marcianus and Mercurius,None,Male,None,Martyr,Martyr,...,None,exiled to Armenia,[],[Arianism],[],[strangled on the orders of the Emperor],[],None,[],Saints Marcianus and Mercurius were martyrs wh...
167,mesra_17_a,Mesra 17,https://www.copticchurch.net/synaxarium/12_17....,"On this day, St. James, the soldier, departed....",St. James the Soldier,None,Male,None,Soldier,Martyr,...,None,None,[],[],"[healing from afflictions, rescue from the sea]","[beating, torture, beheading]",[],None,[],St. James the Soldier was a Christian martyr w...
60,paona_24_a,Paona 24,https://www.copticchurch.net/synaxarium/10_24....,"On this day, St. Moses the Black, whose life s...",St. Moses the Black,Moses,Male,None,Monk,Saint,...,Transformed from a life of sin to become a mon...,None,[],[],[],[],[],Matthew 11:12,[],St. Moses the Black was a former slave and vio...


In [234]:
# Save to one Excel file with different sheets
with pd.ExcelWriter("all_synaxariumscombined.xlsx", engine="openpyxl") as writer:
    result.to_excel(writer, sheet_name="data", index=False)
    allLocations.to_excel(writer, sheet_name="locations", index=False)
    importantVerseReference.to_excel(writer, sheet_name="verses", index=False)
    relationships.to_excel(writer, sheet_name="relationships", index=False)
    spiritualThemes.to_excel(writer, sheet_name="spirital_themes", index=False)
    iconographySymbols.to_excel(writer, sheet_name="iconography_symbols", index=False)
    descriptiveAdjectives.to_excel(writer, sheet_name="descriptive_adjectives", index=False)

In [20]:
import pandas as pd

# Read all sheets into a dict of DataFrames
dfs = pd.read_excel("/Users/maryyoussef/Documents/Synaxarium Project/all_synaxariumscombined.xlsx", sheet_name=None, engine="openpyxl")

# Access each sheet by its name
result = dfs["data"]
allLocations = dfs["locations"]
importantVerseReference = dfs["verses"]
relationships = dfs["relationships"]
spiritualThemes = dfs["spirital_themes"]
iconographySymbols = dfs["iconography_symbols"]
descriptiveAdjectives = dfs["descriptive_adjectives"]


In [29]:
(set(iconographySymbols['iconographySymbols']  ))

{'Altar',
 'Anchor',
 'Angel',
 'Angel Gabriel',
 'Angels',
 'Axe',
 'Bald head (as a symbol of monasticism)',
 'Baptism',
 'Book',
 "Builder's tools",
 'Calf',
 'Camel',
 'Camel hair',
 "Camel's knees",
 'Censer',
 'Chains',
 'Chariot of fire',
 'Cross',
 'Cross of light',
 'Crown',
 'Desert',
 'Dove',
 'Eagle',
 'Fire',
 'Flower',
 'Gospel Book',
 'Gospel book',
 'Grave clothes',
 'Holy Cross',
 'Holy Mary Magdalene',
 'Ivory Bed',
 'Jawbone',
 'Keys',
 'Lamb of God',
 'Lily',
 'Linen cloth',
 'Lion',
 'Long hair',
 'Man',
 'Mantle',
 'Marble coffin',
 'Monastic Robes',
 'Mother',
 'Palm',
 'Palm branches',
 'Raven',
 'Red-hot iron beds',
 'Rock',
 'Scroll',
 'Serpent',
 'Spear',
 'Stag',
 'Star',
 'Stars',
 'Stone',
 'Symbols of preaching',
 'Tongues of Fire',
 'Virgin',
 'Water',
 'Wheel of torture',
 'Wolf',
 'angel',
 'angels',
 'arrows',
 'baptismal font',
 'bears',
 'bible',
 "bishop's pastoral staff",
 'blood',
 'blue robe',
 'boar',
 'broken body',
 'burning bush',
 "camel's 

In [24]:
spiritualThemes['spiritualThemes']

0         asceticism
1       perseverance
2        forgiveness
3            courage
4              faith
            ...     
1374           faith
1375       sacrifice
1376           faith
1377           faith
1378         courage
Name: spiritualThemes, Length: 1379, dtype: object